<a href="https://colab.research.google.com/github/ttogle918/AI_practice/blob/main/%EC%98%A4%ED%83%80%EA%B3%A0%EC%B9%98%EA%B8%B0/typo_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers datasets
!pip install pytorch-transformers transformers
!pip install konlpy
!pip install jamo

In [ ]:
from datasets import load_dataset
dataset_sts = load_dataset('klue', 'sts')
dataset_sts, dataset_sts['train'][0]

Generating train split:   0%|          | 0/11668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/519 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

(DatasetDict({
     train: Dataset({
         features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
         num_rows: 11668
     })
     validation: Dataset({
         features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
         num_rows: 519
     })
 }),
 {'guid': 'klue-sts-v1_train_00000',
  'source': 'airbnb-rtt',
  'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
  'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
  'labels': {'label': 3.7,
   'real-label': 3.714285714285714,
   'binary-label': 1}})

In [ ]:
dataset_sts['train'].num_rows

11668

In [ ]:
import re

# 단어장에 없는 단어 ( UNK ) 확인, model 단어장에 추가

- 맞춤법 오류는 어떻게 할 지?
==> 맞춤법 맞추는 기능 추가?

In [ ]:
from sentence_transformers import SentenceTransformer, models
model_name = 'klue/roberta-base'
embedding_model = models.Transformer(model_name, do_lower_case=True)
pooler = models.Pooling(
      embedding_model.get_word_embedding_dimension(),
      pooling_mode_mean_tokens=True,
      pooling_mode_cls_token=False,
      pooling_mode_max_tokens=False
  )
model = SentenceTransformer(modules=[embedding_model, pooler])

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
def get_UNK(s, li) :
  s = s.replace('.', ' .')    # re.sub형식으로 바꿀 것!   실제 모델링할 때는 띄어쓰기하지 않아도 된다. 알아서 처리됨!!
  s = s.replace(',', ' ,')
  s = s.replace('!', ' !')
  s = s.split()

  copy_string = []
  for ss in li :
    if ss.count('#') > 0 :    # #이 앞에만 있다는 가정.
      copy_string[-1] = copy_string[-1] + ss.replace('#', '')
    else :
      copy_string.append(ss)
  
  try :
    error_string, print_string = '', ''
    i, j = 0, 0
    while i < len(s) and j < len(copy_string):
      if copy_string[i] == '[UNK]' :
        print_string += s[j] + ' '
        i += 1
        j += 1

      elif s[j] == copy_string[i] : 
        i += 1
        j += 1

      else : 
        if s[j].startswith(copy_string[i]) == True :
          i += 1
          copy_string[i] = copy_string[i-1] + copy_string[i] 

        else :
          error_string += s[j]    # 이상한 문자
          j += 1
  except :
    print('error :    ', copy_string)
    print(s)
    print('error : UNK :',  print_string)

  if len(print_string) > 0 :
    print('UNK :',  print_string)

In [ ]:
for splited in ['train', 'validation'] :
  for i in range(dataset_sts[splited].num_rows) :
    sentence1, sentence2 = dataset_sts[splited][i]['sentence1'], dataset_sts[splited][i]['sentence2']
    
    get_UNK(sentence1, model.tokenizer.tokenize(sentence1))
    get_UNK(sentence2, model.tokenizer.tokenize(sentence2))

UNK : 말쑴해 
UNK : 큽니다 
UNK : 삑삑 
error :     ['‘', '‘고마워요', '_', '질병관리본부', '’', '‘', '힘내요', '_', '질병관리본부', '’']
['‘#고마워요_질병관리본부’', '‘#힘내요_질병관리본부’']
error : UNK : 
UNK : 쐬면 
UNK : 쐬면 
UNK : 깼습니다 
UNK : 훠궈집이랑 
UNK : 스케쥴은 
UNK : 헹주등 
UNK : 떙볕에 
UNK : 스케쥴이 
UNK : 훠궈집이요 
UNK : 쐬는 
UNK : 캥거루들의 
UNK : 스케쥴 
error :     ['야', '침실', '무드등을', '[UNK]', '?', '형광등을', '[UNK]', '?']
['야', '침실', '무드등을', '켤래?', '형광등을', '켤래?']
error : UNK : 켤래? 
UNK : 켤래? 
UNK : 스낵 
UNK : 한뼘 
UNK : 큽니다 
UNK : 예쁩니다 
UNK : 어떄? 
UNK : 스케쥴 
UNK : 짰다간 
UNK : 뜹니까? 
UNK : 켤꺼야 켤꺼야? 
error :     ['측정은', '권장', '실내', '라돈', '기준', '(', '(148', 'Bq', 'Bq/', 'Bq/[UNK]', '미만', ')', '준수', '여부를', '확인하는', '방법입니다', '.']
['측정은', '권장', '실내', '라돈', '기준', '(148', 'Bq/μ', '미만)', '준수', '여부를', '확인하는', '방법입니다', '.']
error : UNK : 
UNK : 스케쥴 
UNK : 콸콸 
UNK : 어댑터 
UNK : 켤때 
UNK : 뵐 
UNK : 슬펐어요 
error :     ['세계문화예술교육', '주간과', '‘', '‘꿈의', '오케스트라', '오케스트라’', '10주년을', '맞아', '인형', '‘', '‘[UNK]', '’', '와', '가수', '‘', '헨리', '’', '가', '문화예술교육의', '인식을', '확산하는'

In [ ]:
model.tokenizer.add_tokens(['핼로윈데이', '오믈렛', '슬펐', '슬픕', '헹굼', '헹구', '뺄', '켤', '챗봇', '예쁩', '스케쥴', '찝찝', '갭투자', '초코', '케익', '추천', '버젼', '펍', '뜹', '니까?', '땡볕', '쨍쨍', '여쭙', '켤레', '멎', '뺄', '샐', '캥거루', '훠궈', '뼘', '한', '삑삑'])


26

In [ ]:
# 남은 UNK 확인 ==> 거의 맞춤법 문제
for splited in ['train', 'validation'] :
  for i in range(dataset_sts[splited].num_rows) :
    sentence1, sentence2 = dataset_sts[splited][i]['sentence1'], dataset_sts[splited][i]['sentence2']
    
    get_UNK(sentence1, model.tokenizer.tokenize(sentence1))
    get_UNK(sentence2, model.tokenizer.tokenize(sentence2))

UNK : 말쑴해 
UNK : 큽니다 
error :     ['‘', '‘고마워요', '_', '질병관리본부', '’', '‘', '힘내요', '_', '질병관리본부', '’']
['‘#고마워요_질병관리본부’', '‘#힘내요_질병관리본부’']
error : UNK : 
UNK : 쐬면 
UNK : 쐬면 
UNK : 깼습니다 
UNK : 헹주등 
UNK : 떙볕에 
UNK : 쐬는 
UNK : 스낵 
UNK : 큽니다 
UNK : 어떄? 
UNK : 짰다간 
error :     ['측정은', '권장', '실내', '라돈', '기준', '(', '(148', 'Bq', 'Bq/', 'Bq/[UNK]', '미만', ')', '준수', '여부를', '확인하는', '방법입니다', '.']
['측정은', '권장', '실내', '라돈', '기준', '(148', 'Bq/μ', '미만)', '준수', '여부를', '확인하는', '방법입니다', '.']
error : UNK : 
UNK : 콸콸 
UNK : 어댑터 
UNK : 뵐 
error :     ['세계문화예술교육', '주간과', '‘', '‘꿈의', '오케스트라', '오케스트라’', '10주년을', '맞아', '인형', '‘', '‘[UNK]', '’', '와', '가수', '‘', '헨리', '’', '가', '문화예술교육의', '인식을', '확산하는', '캠페인에', '동참한다', '.']
['세계문화예술교육', '주간과', '‘꿈의', '오케스트라’', '10주년을', '맞아', '인형', '‘펭수’와', '가수', '‘헨리’가', '문화예술교육의', '인식을', '확산하는', '캠페인에', '동참한다', '.']
error : UNK : 
UNK : 카디쾨이 
UNK : 큽니까? 
UNK : 지킵시다 
UNK : 쇤부른 
error :     ['지난', '3년', '동안', ',', '12월과', '3월', '사이의', '평균', '나쁜', '일수는', '33일이었고', ',', '초미세먼지의', '평균